In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget -q https://drive.google.com/drive/folders/1azljebZHymH6ZYeFC8Hfa_jBTj6BlsXd?usp=sharing

In [ ]:
!unzip -q "/content/drive/My Drive/Dataset/Covid5-20201009T145448Z-001.zip"

In [ ]:
!ls Covid5/Covid5/train/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder,DatasetFolder
from torch.utils.data import DataLoader

from torch.utils.tensorboard import SummaryWriter

from google.colab import files

from tqdm.notebook import tqdm

from PIL import Image
import os
import warnings
warnings.filterwarnings('ignore')
%load_ext tensorboard

device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

In [ ]:
for covid in os.listdir('Covid5/Covid5/train/covid'):
  Covid_path = os.path.join('Covid5/Covid5/train/covid',covid)
  if os.path.splitext(Covid_path)[-1] == '.jpeg , .png , .jpg':
    try:
      Image.open(Covid_path)
    except:
      print('Delete:{}'.format(Covid_path))
      os.remove(Covid_path)

In [ ]:
for non in os.listdir('Covid5/Covid5/train/non'):
  Normal_path = os.path.join('Covid5/Covid5/train/non',non)
  if os.path.splitext(Normal_path)[-1] == '.jpeg , .png , .jpg':
    try:
      Image.open(Normal_path)
    except:
      print('Delete:{}'.format(Normal_path))
      os.remove(Normal_path)

In [ ]:
Data_Dir = "Covid5/Covid5/train//"

In [ ]:
preprocess = transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485,0.456,0.406],
                                                      std=[0.229,0.224,0.225])
])
def img_loader(x):
  img = Image.open(x)
  try:
    img = img.convert('RGB')
  except:
    pass
  finally:
    return img
    
dataset = DatasetFolder('Covid5/Covid5/train',transform=preprocess,
                        extensions=('jpeg','png','jpg'),
                        loader=img_loader)
print(dataset)
print(dataset.classes)
print(dataset.class_to_idx)

In [ ]:
dataloader = DataLoader(dataset,
                        batch_size=8,
                        shuffle=True)
images,labels = next(iter(dataloader))
print(images.size())
print(labels.size())

In [ ]:
def unnormalize(images):
  mean = torch.tensor([0.485,0.456,0.406]).view(1,3,1,1)
  std = torch.tensor([0.229,0.224,0.225]).view(1,3,1,1)
  images = images * std + mean
  return images

images = unnormalize(images)
img_grid = torchvision.utils.make_grid(images)

!rm -rf runs
writer = SummaryWriter('runs/data-viz')
writer.add_image(str([dataset.classes[i] for i in labels]),img_grid)
writer.close

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs

In [ ]:
def load_model():
  model = torch.hub.load('pytorch/vision:v0.6.0','resnet18',pretrained=True)

  for param in model.parameters():
    param.requires_grad=False


  in_ftrs = model.fc.in_features
  model.fc = nn.Linear(in_ftrs,2)

  return model

In [ ]:
resnet = load_model().to(device)

writer = SummaryWriter('runs/model-vlz')
writer.add_graph(resnet,images.to(device))
writer.close()

In [ ]:
def images_to_probs(net,images):
  '''generates prediction and corresponding probabilities from a trained network and a list of images'''
  output = net(images).cpu()
  #convert output probabilities to predicted class

  _,preds_tensor = torch.max(output,1)
  preds=np.squeeze(preds_tensor.numpy())
  return preds,[F.softmax(el,dim=0)[i].item() for i,el in zip(preds,output)]

def evaluate(net,dataloader):
  total,correct= 0,0

  model.eval()
  for batch in dataloader:
    images,labels = batch
    images,labels = images.to(device),labels.to(device)

    outs = model(images)
    out_labels = torch.argmax(outs,axis=1)

    total += labels.size(0)
    correct += (labels == out_labels).sum().item()
    return 100 * correct/total

In [ ]:
def train(net,loss_fn,opt,dataloader,writer,epochs):
  num_steps = 0
  min_loss = 1e+10

  if os.path.isdir('checkpoints'):
    !rm -rf checkpoints/
  os.mkdir('checkpoints')

  for epoch in tqdm(range(1,epochs+1),total=epochs,desc='training'):
    running_loss = []
    net.train()
    for images,labels in dataloader:
      images,labels = images.to(device),labels.to(device)
      num_steps +=1

      outs = net(images)
      loss = loss_fn(outs,labels)

      running_loss.append(loss.item())
      writer.add_scalar('step loss',loss.item(),num_steps)
      opt.zero_grad()
      loss.backward()
      opt.step()
      del images,labels,outs
      torch.cuda.empty_cache()

    epoch_loss = sum(running_loss)/len(running_loss)
    acc = evaluate(net,dataloader)
    writer.add_scalar('Epoch loss',epoch_loss,epoch)
    writer.add_scalar('accuracy',acc,epoch)
    if epoch_loss < min_loss:
      min_loss = epoch_loss
      torch.save(net.state_dict(),
                  'checkpoints/model_L{:2f}.pt'.format(min_loss))
      writer.add_scalar('checkpoints',min_loss,epochs)
  print('training finished')



In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs

model=load_model().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(),lr=0.001)

!rm -rf runs/lr_0.001-exp1

writer = SummaryWriter('runs/lr_0.001-exp1')
train(model,loss_fn,opt,dataloader,writer,10)
writer.close()

In [ ]:
model_name = min(os.listdir('checkpoints'))
print(model_name)

In [ ]:
os.listdir('checkpoints')

In [ ]:
files.download(os.path.join('checkpoints',model_name))

In [ ]:
!cp checkpoints/model_L0.075021.pt model_L0.075021.pt